In [1]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Lambda, Cropping2D
from tensorflow.keras.optimizers import Adam

# Cargar el archivo CSV sin encabezados
data_path = '/home/mitnik/Documents/MNA/autonomous_driving/proyecto_final/angle.csv'  # Cambia esto por la ruta correcta de tu archivo CSV
image_dir = '/home/mitnik/Documents/MNA/autonomous_driving/proyecto_final/image_bank'  # Cambia esto por la ruta correcta de tu directorio de imágenes
df = pd.read_csv(data_path, header=None)

# Asignar nombres de columnas manualmente
df.columns = ['image', 'steering_angle']

# Preprocesar las imágenes y los ángulos
images = []
angles = []

for index, row in df.iterrows():
    img_path = os.path.join(image_dir, row[0])
    print(img_path)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128, 64))  # Asegúrate de que todas las imágenes tengan el mismo tamaño
    images.append(img)
    angles.append(row[1])

images = np.array(images)
angles = np.array(angles)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, angles, test_size=0.2, random_state=42)

# Definir el modelo basado en la arquitectura NVIDIA
def create_model():
    model = Sequential([
        Lambda(lambda x: x / 255.0 - 0.5, input_shape=(64, 128, 3)),  # Normalización y tamaño de imagen ajustado
        Conv2D(24, (5, 5), activation='relu', strides=(2, 2)),
        Conv2D(36, (5, 5), activation='relu', strides=(2, 2)),
        Conv2D(48, (5, 5), activation='relu', strides=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.5),
        Dense(50, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='mse')
    return model

model = create_model()

# Configurar data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = datagen.flow(X_train, y_train, batch_size=32).repeat()
validation_generator = datagen.flow(X_test, y_test, batch_size=32).repeat()

# Calcular steps_per_epoch y validation_steps
batch_size = 32
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_test) // batch_size

# Entrenar el modelo
history = model.fit(train_generator, epochs=30, validation_data=validation_generator, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)

# Guardar el modelo
model.save('model.h5')


2024-06-23 23:16:49.623568: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_374626/2743595106.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_path = os.path.join(image_dir, row[0])
/tmp/ipykernel_374626/2743595106.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  angles.append(row[1])
[ WARN:0@53.598] global loadsave.cpp:241 findDecoder imread_('/home/mitnik/Documents/MNA/autonomous_driving/proyecto_final/image_bank/2024-06-23 03-50-29.png'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
